In [ ]:
import tensorflow as tf
from tensorflow.python.client import timeline
from tensorflow.python.client import device_lib 

In [ ]:
print(device_lib.list_local_devices())

In [11]:
n = 10000

'''with tf.device("cpu:0"):
    a1 = tf.random_uniform(shape=[n,n], minval=0, maxval=10, dtype=tf.float32, seed=312, name="A1")
    a2 = tf.random_uniform(shape=[n,n], minval=0, maxval=10, dtype=tf.float32, seed=312, name="A2")
    a3 = tf.matmul(a1, a2, name='A3')

with tf.device("gpu:0"):
    a4 = tf.random_uniform(shape=[n,n], minval=0, maxval=10, dtype=tf.float32, seed=312, name="A4")
    a5 = tf.random_uniform(shape=[n,n], minval=0, maxval=10, dtype=tf.float32, seed=312, name="A5")
    a6 = tf.matmul(a4, a5, name='A6')'''
    
    
x = tf.random_normal([n, n], name='X')
y = tf.random_normal([n, n], name='Y')
res = tf.matmul(x, y, name='Result')
    

In [12]:
# turn off graph optimizations
no_opt = tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0,
                             do_common_subexpression_elimination=False,
                             do_function_inlining=False,
                             do_constant_folding=False)
config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=no_opt),
                        log_device_placement=True,
                        allow_soft_placement=False,
                        device_count={"CPU": 8},
                        inter_op_parallelism_threads=3,
                        intra_op_parallelism_threads=1)
sess = tf.Session(config=config)

metadata = tf.RunMetadata()
options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE,
                        output_partition_graphs=True)

#sess.run([a3.op, a6.op], options=options, run_metadata=metadata)
sess.run(res.op, options=options, run_metadata=metadata)

trace = timeline.Timeline(step_stats=metadata.step_stats)
with open('/tmp/timeline.json', 'w') as out:
    out.write(trace.generate_chrome_trace_format())

print(str(metadata))

step_stats {
  dev_stats {
    device: "/job:localhost/replica:0/task:0/gpu:0"
    node_stats {
      node_name: "_SOURCE"
      all_start_micros: 1499105535684576
      op_start_rel_micros: 3
      op_end_rel_micros: 8
      all_end_rel_micros: 15
      memory {
        allocator_name: "GPU_0_bfc"
      }
      timeline_label: "_SOURCE = NoOp()"
      scheduled_micros: 1499105535684562
      memory_stats {
      }
    }
    node_stats {
      node_name: "X_1/shape"
      all_start_micros: 1499105535684597
      op_start_rel_micros: 1
      op_end_rel_micros: 2
      all_end_rel_micros: 11
      memory {
        allocator_name: "GPU_0_bfc"
      }
      output {
        tensor_description {
          dtype: DT_INT32
          shape {
            dim {
              size: 2
            }
          }
          allocation_description {
            requested_bytes: 8
            allocated_bytes: 256
            allocator_name: "cuda_host_bfc"
            allocation_id: 27
            ptr: 

In [13]:
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(res, options=options, run_metadata=metadata)

print (metadata.partition_graphs)